In [3]:
import os
import pandas as pd

# --- PARAMETERS ---
PARAMS = {
    'car_emissions_kg_per_km': 0.22,
    'bike_emissions_kg_per_km': 0.05,
    'auto_emissions_kg_per_km': 0.07,
    'car_cost_inr_per_km': 12,
    'bike_cost_inr_per_km': 2.5,
    'auto_cost_inr_per_km': 15,
    'auto_time_factor': 1.33,
}

# --- INPUT & OUTPUT DIRECTORIES ---
FACILITY_TYPES = ['chc', 'hsc', 'phc']
INPUT_BASE = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts_Car"
OUTPUT_BASE = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Enriched_CarOutputs"

# DH/SDH (district hospital/sub-district hospital) directories
DH_INPUT = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\routing_dh"
DH_OUTPUT = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Enriched_CarOutputs\dh"
os.makedirs(DH_OUTPUT, exist_ok=True)

# --- PROCESS CHC/HSC/PHC FACILITY TYPES ---
for fac in FACILITY_TYPES:
    input_folder = os.path.join(INPUT_BASE, fac)
    output_folder = os.path.join(OUTPUT_BASE, fac)
    os.makedirs(output_folder, exist_ok=True)

    for fname in os.listdir(input_folder):
        if fname.endswith('.csv'):
            fpath = os.path.join(input_folder, fname)
            df = pd.read_csv(fpath)

            # Standardize column names for consistency
            df.columns = [col.lower().strip().replace(' ', '_') for col in df.columns]

            # Enrichment (for standard files)
            if 'travel_time_min' in df.columns and 'distance_km' in df.columns:
                df['travel_time_bike'] = df['travel_time_min']
                df['travel_time_auto'] = df['travel_time_min'] * PARAMS['auto_time_factor']

                df['car_emissions_kg'] = df['distance_km'] * PARAMS['car_emissions_kg_per_km']
                df['bike_emissions_kg'] = df['distance_km'] * PARAMS['bike_emissions_kg_per_km']
                df['auto_emissions_kg'] = df['distance_km'] * PARAMS['auto_emissions_kg_per_km']

                df['car_cost_inr'] = df['distance_km'] * PARAMS['car_cost_inr_per_km']
                df['bike_cost_inr'] = df['distance_km'] * PARAMS['bike_cost_inr_per_km']
                df['auto_cost_inr'] = df['distance_km'] * PARAMS['auto_cost_inr_per_km']

                df.to_csv(os.path.join(output_folder, fname), index=False)
                print(f"✅ {fname} enriched and saved to {output_folder}")
            else:
                print(f"❌ {fname} missing expected columns for {fac} (skipped).")

# --- PROCESS DH/SDH FILES (SPECIAL COLUMNS) ---
for fname in os.listdir(DH_INPUT):
    if fname.endswith('.csv'):
        fpath = os.path.join(DH_INPUT, fname)
        df = pd.read_csv(fpath)

        # Standardize column names for consistency
        df.columns = [col.lower().strip().replace(' ', '_') for col in df.columns]

        # Check if required columns for DH/SDH exist
        if 't_dh_sdh_min' in df.columns and 'd_dh_sdh_km' in df.columns:
            df['travel_time_bike'] = df['t_dh_sdh_min']
            df['travel_time_auto'] = df['t_dh_sdh_min'] * PARAMS['auto_time_factor']

            df['car_emissions_kg'] = df['d_dh_sdh_km'] * PARAMS['car_emissions_kg_per_km']
            df['bike_emissions_kg'] = df['d_dh_sdh_km'] * PARAMS['bike_emissions_kg_per_km']
            df['auto_emissions_kg'] = df['d_dh_sdh_km'] * PARAMS['auto_emissions_kg_per_km']

            df['car_cost_inr'] = df['d_dh_sdh_km'] * PARAMS['car_cost_inr_per_km']
            df['bike_cost_inr'] = df['d_dh_sdh_km'] * PARAMS['bike_cost_inr_per_km']
            df['auto_cost_inr'] = df['d_dh_sdh_km'] * PARAMS['auto_cost_inr_per_km']

            df.to_csv(os.path.join(DH_OUTPUT, fname), index=False)
            print(f"✅ {fname} enriched and saved to {DH_OUTPUT}")
        else:
            print(f"❌ {fname} missing DH/SDH time/distance columns (skipped).")

print("🎉 All files processed and enriched.")

✅ anugul_village_to_chc.csv enriched and saved to C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Enriched_CarOutputs\chc
✅ balangir_village_to_chc.csv enriched and saved to C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Enriched_CarOutputs\chc
✅ baleshwar_village_to_chc.csv enriched and saved to C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Enriched_CarOutputs\chc
✅ bargarh_village_to_chc.csv enriched and saved to C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Enriched_CarOutputs\chc
✅ bhadrak_village_to_chc.csv enriched and saved to C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Enriched_CarOutputs\chc
✅ boudh_village_to_chc.csv enriched and saved to C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Enriched_C

In [4]:
# ------------------------------------------------------------------------------
# STEP-BY-STEP PROCESS: Travel, Emissions & Cost Enrichment for Odisha Health Access
# ------------------------------------------------------------------------------
# 1. Prepare Inputs
#    - Collect OSRM routing outputs for each district and facility type (HSC, PHC, CHC, DH/SDH).
#    - Ensure standard columns in input files:
#      * HSC/PHC/CHC: ['lgd_villagecode', 'village_name', 'district', 'nin', 'facility_name', 
#                      'facility_type', 'travel_time_min', 'distance_km', ...]
#      * DH/SDH: ['village_id', ..., 't_dh_sdh_min', 'd_dh_sdh_km', ...]
#
# 2. Define Mode Parameters
#    - Set per-km emission and cost factors (from Indian transport literature):
#        Car:   0.22 kg CO₂/km,  ₹12/km
#        Bike:  0.05 kg CO₂/km,  ₹2.5/km
#        Auto:  0.07 kg CO₂/km,  ₹15/km
#    - Set auto travel time multiplier: 1.33 × car time (to reflect lower average speed)
#
# 3. Batch Processing (by facility type and district)
#    - For HSC/PHC/CHC files:
#        a. For each file, read data.
#        b. Calculate derived columns for every row:
#             travel_time_bike    = travel_time_min
#             travel_time_auto    = travel_time_min × 1.33
#             car_emissions_kg    = distance_km × 0.22
#             bike_emissions_kg   = distance_km × 0.05
#             auto_emissions_kg   = distance_km × 0.07
#             car_cost_inr        = distance_km × 12
#             bike_cost_inr       = distance_km × 2.5
#             auto_cost_inr       = distance_km × 15
#        c. Save enriched file to output folder for the facility type.
#    - For DH/SDH files:
#        a. For each file, read data.
#        b. Use t_dh_sdh_min as travel time, d_dh_sdh_km as distance.
#        c. Calculate derived columns for every row:
#             travel_time_bike    = t_dh_sdh_min
#             travel_time_auto    = t_dh_sdh_min × 1.33
#             car_emissions_kg    = d_dh_sdh_km × 0.22
#             bike_emissions_kg   = d_dh_sdh_km × 0.05
#             auto_emissions_kg   = d_dh_sdh_km × 0.07
#             car_cost_inr        = d_dh_sdh_km × 12
#             bike_cost_inr       = d_dh_sdh_km × 2.5
#             auto_cost_inr       = d_dh_sdh_km × 15
#        d. Save enriched file to DH output folder.
#
# 4. Validation
#    - Check output files for presence of all expected columns and correct calculation per row.
#    - Ensure naming and order is consistent across all facility types.
#
# 5. Ready for Downstream Analysis
#    - These enriched outputs are now ready for further merging (population, telemedicine, etc.), scenario modeling, or statistical analysis.
#
# ------------------------------------------------------------------------------
# End of Process Documentation
# ------------------------------------------------------------------------------
